# TASK 1 : Install Dependencies & Run a SparkSession


In [2]:
#install pyspark

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 34.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=339c23d64f635297d7aa5d094fdbb336e1202e6b9cb9005988af92337a423f6f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
from pyspark.sql.session import SparkSession
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark').getOrCreate()

# TASK 2 : Clone & Explore dataset

In [5]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
Unpacking objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0


In [8]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [9]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header=True,inferSchema=True)

In [10]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [11]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [12]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [13]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [14]:
#drop the unnecessary column
df  = df.drop('Serial No')

In [15]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [17]:
#check for null values
for i in df.columns:
  print(i+":", df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [20]:
# correlation analysis
for col in df.columns:
  print('correlation to chance of admit col for {} is{}'.format(col,df.stat.corr('Chance of Admit',col)))

correlation to chance of admit col for GRE Score is0.8103506354632598
correlation to chance of admit col for TOEFL Score is0.7922276143050823
correlation to chance of admit col for University Rating is0.6901323687886892
correlation to chance of admit col for SOP is0.6841365241316723
correlation to chance of admit col for LOR is0.6453645135280112
correlation to chance of admit col for CGPA is0.882412574904574
correlation to chance of admit col for Research is0.5458710294711379
correlation to chance of admit col for Chance of Admit is1.0


In [21]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],outputCol='features')

In [22]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [24]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','Chance of Admit')

In [25]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [26]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7,0.3])

In [28]:
, LinearRegressionModel
#build & train the model
models = LinearRegression(featuresCol='features',labelCol='Chance of Admit')
model = models.fit(train)

In [29]:
#get coefficients & intercept
print("coefficients:",model.coefficients)
print('intercept:', model.intercept)

coefficients: [0.002417314851062564,0.003768486948450711,0.14132343038021833]
intercept: -1.6604206246635818


In [33]:
#get summary of the model
summary = model.summary

In [35]:
#print the rmse & r2 score
print('RMSE', summary.rootMeanSquaredError)
print('r2 score', summary.r2)

RMSE 0.06229012295257466
r2 score 0.8055575239233355


# TASK 6 : Evaluate & Save the Model

In [36]:
#transform on the test data
predictions = model.transform(test)

In [37]:
#display the predictions
predictions.show(20)

+------------------+---------------+------------------+
|          features|Chance of Admit|        prediction|
+------------------+---------------+------------------+
|  [293.0,97.0,7.8]|           0.64|0.5157186176631712|
| [294.0,93.0,7.36]|           0.46|0.4408796753531352|
| [294.0,95.0,7.64]|           0.49|0.4879872097564977|
| [295.0,99.0,7.65]|           0.57|0.5068917067051653|
| [296.0,99.0,7.28]|           0.47| 0.457019352315547|
| [297.0,98.0,7.67]|           0.59|0.5107843180664442|
| [297.0,100.0,7.9]|           0.52|0.5508256809507956|
| [298.0,97.0,7.21]|           0.45|0.4444243679941555|
|[298.0,101.0,7.69]|           0.53|0.5273335623704634|
| [299.0,94.0,7.34]|           0.42|0.4539082679492943|
|[299.0,100.0,7.88]|           0.51|0.5528338420453165|
|[299.0,100.0,7.88]|           0.68|0.5528338420453165|
|[299.0,100.0,7.89]|           0.59|0.5542470763491185|
| [299.0,106.0,8.4]|           0.64|0.6489329475337342|
|  [300.0,97.0,8.1]|           0.65|0.5750368507

In [38]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol= 'prediction',labelCol='Chance of Admit', metricName = 'r2')
print('r2 on the test data', evaluator.evaluate(predictions))

r2 on the test data 0.7979094041156226


In [39]:
#save the model
model.save("model")

In [40]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('model')